### Partial Derivative of a Single Neuron
A single neuron with 3 inputs can be written in function form as 

ReLU(sum( mul(x_0, w_0) , mul(x_1, w_1), mul(x_2, w_2), b))

The partial derivative of this output with respect to s a specific weight (let's say w_0) will be calculated through the chain rule

The ReLU function, which outputs z, if z is more than 0, has the derivative 1 if z is more than 0, and 0 otherwise

In [11]:
### Foward Pass
x = [1.0, -2.0, 3.0] # input values from the 3 neurons in the preceding layer
w = [-3.0, -1.0, 2.0] # weights
b = 1.0 # bias

# multiplying inputs by weights
xw0 = x[0] * w[0]
xw1 = x[1] * w[1]
xw2 = x[2] * w[2]

# adding the weighted inputs and a bias
z = xw0 + xw1 + xw2 + b

# ReLU activatin function
relu = max(z, 0)

### Backwards pass

# the derivate from the next layer
dvalue = 1.0

# derivative of the ReLU and the chain rule
drelu_dz = dvalue * (1. if z > 0 else 0.)
print("drelu_dz:", drelu_dz)

# partial derivatives of the multiplication
# they're all 1s because the derivative of
# a summation function is always 1
dsum_dxw0 = 1
dsum_dxw1 = 1
dsum_dxw2 = 1
dsum_db = 1

# their chain rules
drelu_dxw0 = drelu_dz * dsum_dxw0
drelu_dxw1 = drelu_dz * dsum_dxw1
drelu_dxw2 = drelu_dz * dsum_dxw2
drelu_db   = drelu_dz * dsum_db
print(drelu_dxw0, drelu_dxw1, drelu_dxw2, drelu_db)


# partial derivates of the multiplications
# for a multiplication function, f(x, y) = xy
# the partial derivative of the function w.r.t
# either of the multiplicants is always the other
# multiplicant, hence:
dmul_dx0 = w[0]
dmul_dx1 = w[1]
dmul_dx2 = w[2]

dmul_dw0 = x[0]
dmul_dw1 = x[1]
dmul_dw2 = x[2]

#their chain rules
drelu_dx0 = drelu_dxw0 * dmul_dx0
drelu_dw0 = drelu_dxw0 * dmul_dw0
drelu_dx1 = drelu_dxw1 * dmul_dx1
drelu_dw1 = drelu_dxw1 * dmul_dw1
drelu_dx2 = drelu_dxw2 * dmul_dx2
drelu_dw2 = drelu_dxw2 * dmul_dw2

print(drelu_dx0, drelu_dw0, drelu_dx1, drelu_dw1, drelu_dx2, drelu_dw2)

drelu_dz: 1.0
1.0 1.0 1.0 1.0
-3.0 1.0 -1.0 -2.0 2.0 3.0


#### Simplifications

In [8]:
drelu_dz = dvalue * (1. if z > 0 else 0.)

dsum_dxw0 = 1
drelu_0 = drelu_dz * dsum_dxw0

dmul_dz0 = w[0]
drelu_dz0 = drelu_dxw0 * dmul_dx0

Can be simplified down to

In [ ]:
drelu_dz0 = dvalue * (1. if z > 0 else 0.) * w[0]

All together, these final partial derivatives make up our gradient

In [10]:
dx = [drelu_dx0, drelu_dx1, drelu_dx2]
dw = [drelu_dw0, drelu_dw1, drelu_dw2]
db = drelu_db

The dw and db gradients will be used to update our weights and biases in an intelligent manner to decrease the output of the loss function. This can be done by adding a negative fraction of this gradient to our weights and biases.

The dx gradient will be passed down to the previous layer, so it can do the same calculations and alter its weights and biases accordingly

### Partial Derivative of a Layer of Neurons
Each neuron in the layer gives its output to every neuron in the next layer, so every neuron in the next layer will be giving each neuron in this layer a partial derivative with respect to its output. So each neuron in this layer receives an array of partial derivates from the next layer. Since each value in this array represents how much the loss function changes per unit increase in the output of this individual neuron, we sum this entire array and work with that.

The partial derivates w.r.t an input, the corresponding weight is multiplied with the chaining partial derivative. And likewise, the partial derivatives of w.r.t a weight, the corresponding input is multiplied with the chaining partial derivative.

In the following example, we have 3 neurons, but with 4 inputs, so we have 3 sets of 4 weights, preparing the partial derivatives to send to the previous layer

In [13]:
import numpy as np

# passeed in gradient from the next layer
dvalues = np.array([[1., 1., 1.,]])
# is actually an array of arrays, because of the batches
# but for now, we'll only be concerned with the 0th element

# 3 sets of weights, one for each neuron
# 4 inputs, thus 4 weights
# recall the weights are kept transposed
weights = np.array([[0.2, 0.8, -0.5, 1],
                    [0.5, -0.91, 0.26, -0.5],
                    [-0.26, -.027, 0.17, 0.87]]).T

# sum weights of given input and multiply
# by the passed in gradient for this neuron
dx0 = sum(weights[0] * dvalues[0])
dx1 = sum(weights[1] * dvalues[0])
dx2 = sum(weights[2] * dvalues[0])
dx3 = sum(weights[3] * dvalues[0])

# the gradient of the neuron with respect to the inputs
dinputs = np.array([dx0, dx1, dx2, dx3])
# it will be passed along into the previous layer

print(dinputs)

[ 0.44  -0.137 -0.07   1.37 ]


The same operations, but done through the numpy dot product

In [15]:
import numpy as np

# passeed in gradient from the next layer
dvalues = np.array([[1., 1., 1.,]])
# is actually an array of arrays, because of the batches
# but for now, we'll only be concerned with the 0th element

# 3 sets of weights, one for each neuron
# 4 inputs, thus 4 weights
# recall the weights are kept transposed
weights = np.array([[0.2, 0.8, -0.5, 1],
                    [0.5, -0.91, 0.26, -0.5],
                    [-0.26, -.027, 0.17, 0.87]]).T

# sum weights of given input and multiply
# by the passed in gradient for this neuron
dinputs = np.dot(dvalues[0], weights.T)


print(dinputs)

[ 0.44  -0.137 -0.07   1.37 ]


And finally, handling a batch of samples (more items in the dvalues array). Nothing needs to be changed in the code

In [5]:
import numpy as np

# passeed in gradient from the next layer
dvalues = np.array([[1., 1., 1.,],
                    [2., 2., 2.,],
                    [3., 3., 3.,]])

# 3 sets of weights, one for each neuron
# 4 inputs, thus 4 weights
# recall the weights are kept transposed
weights = np.array([[0.2, 0.8, -0.5, 1],
                    [0.5, -0.91, 0.26, -0.5],
                    [-0.26, -.027, 0.17, 0.87]]).T

# sum weights of given input and multiply
# by the passed in gradient for this neuron
dinputs = np.dot(dvalues, weights.T)


print(dinputs)

[[ 0.44  -0.137 -0.07   1.37 ]
 [ 0.88  -0.274 -0.14   2.74 ]
 [ 1.32  -0.411 -0.21   4.11 ]]


#### Gradients with respect to weights
Very similar to calculating w.r.t inputs. Just chain the inputs instead of the weights

In [4]:
import numpy as np

# passeed in gradient from the next layer
dvalues = np.array([[1., 1., 1.,],
                    [2., 2., 2.,],
                    [3., 3., 3.,]])

# 3 3 sets of inputs - samples
inputs = np.array([[1, 2, 3, 2.5],
                   [2., 5., -1., 2],
                   [-1.5, 2.7, 3.3, -0.8]])

# sum weights of given input and multiply
# by the passed in gradient for this neuron
dweights = np.dot(inputs.T, dvalues)


print(dweights)

[[ 0.5  0.5  0.5]
 [20.1 20.1 20.1]
 [10.9 10.9 10.9]
 [ 4.1  4.1  4.1]]


The output shape matches the weights, rather than the dvalues, becase we summed the inputs for each weight then multiplied them by the input gradient. The dweights will now be used to update the weights

#### Gradients with respect to biases
Since the biases are only used in the sum operation, and the derivative of the sum is always equal to 1, which is multiplied to the chaining gradient, we just have to sum the list of gradients, to combine the gradient from all the samples

In [7]:
import numpy as np

# passeed in gradient from the next layer
dvalues = np.array([[1., 1., 1.,],
                    [2., 2., 2.,],
                    [3., 3., 3.,]])

# one bias for each neuron
# biases are the row vector with a shape (1, neurons)
biases = np.array([[2, 3, 0.5]])

# dbiases - sum values, over the samples (first axis),
# and keepdims since this by default will produce a plain list
dbiases = np.sum(dvalues, axis = 0, keepdims =True)

print(dbiases)

[[6. 6. 6.]]


#### Gradient with respect to the ReLU
When chained, basically means if the input into the ReLU is more than 0, return the chaining gradient so far, and else return 0

In [11]:
import numpy as np

# example layer output
z = np.array([[1, 2, -3 ,-4],
              [2, -7, -1, 3],
              [-1, 2, 5, -1]])

dvalues = np.array([[1, 2, 3, 4],
                    [5, 6, 7, 8],
                    [9, 10, 11, 12]])

# ReLU activations's derivative
drelu = np.zeros_like(z)
drelu[z > 0] = 1

print(drelu)

# its chain rule
drelu *= dvalues

print(drelu)

[[1 1 0 0]
 [1 0 0 1]
 [0 1 1 0]]
[[ 1  2  0  0]
 [ 5  0  0  8]
 [ 0 10 11  0]]


##### Simplified

In [12]:
import numpy as np

# example layer output
z = np.array([[1, 2, -3 ,-4],
              [2, -7, -1, 3],
              [-1, 2, 5, -1]])

dvalues = np.array([[1, 2, 3, 4],
                    [5, 6, 7, 8],
                    [9, 10, 11, 12]])

drelu = dvalues.copy()
drelu[z <= 0] = 0

print(drelu)

[[ 1  2  0  0]
 [ 5  0  0  8]
 [ 0 10 11  0]]


#### Put together into our class
To calculate the partial derivate with respect to the weight, our Dense Layer needs to remember the input, we make the appropriate alteration

Meanwhile, for backpropagation, the partial derivative will be passed back, all the way from the loss function

A layer receives the partial derivative, and uses it to calculate the partial derivative of the loss function with respect to the weight, and using that gradient to update the weights, and with respect to the inputs, and passes it along backwards


In [13]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

class Layer_Dense:
    
    # layer initialization
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
    # forward pass
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases
        self.inputs = inputs
        
    # backward pass
    def backward(self, dvalues):
        # dvalues is the partial derivate of the loss function
        # with respect to the next functin in the sequence, in
        # this case being the ReLU
        
        # every neuron in the next layer is giving us a partial derivative
        # for every input from every neuron in this layer, so we're receiving
        # a 2D array, but also because of the batches, we're receiving a 3D array
        
        # gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis = 0, keepdims = True)
        
        # gradient on values to pass further back
        self.dinputs = np.dot(dvalues, self.weights.T)

# same for the ReLU class
# Rectified Linear Unit Activation Function
class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)
    
    # backward pass
    def backward(self, dvalues):
        # since we need to modify the original variable,
        # we make a copy so we keep the original around
        self.dinputs = dvalues.copy()
        
        # zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0
        


In [ ]:


        
class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis = 1, keepdims = True))
        probabilities = exp_values / np.sum(exp_values, axis = 1, keepdims = True)
        self.output = probabilities



### Categorical Cross-Entropy Loss Derivative
The partial derivative of the loss with respect to the predicted value, y, boils down to the negative real answer divided by the predicted answer

In [15]:
class Loss:
    def calculate(self, outputs, y):
        # y is the intended target values
        sample_losses = self.forward(outputs, y)
        data_loss = np.mean(sample_losses)
        return data_loss
    
class Loss_CategoricalCrossEntropy(Loss):
    #inheriting from the base Loss class
    def forward(self, y_pred, y_true):
        # y_pred will come from the neural network
        # y_true will come from the training set
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        
        if len(y_true.shape) == 1:
            # means scalar class values have been passed
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            # one hot encoded values have been passed
            correct_confidences = np.sum(y_pred_clipped * y_true, axis = 1)
            
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    
    # Backwards pass
    def backward(self, dvalues, y_true):
        # number of samples
        samples = len(dvalues)
        # number of labels in every sample
        # we'll use the first sample to count them
        label = len(dvalues[0])
        
        # if labels are sparse, turn them into one-hot vector
        # in case the shape is as [3, 0, 2] etc etc per sample
        # we turn them into one-hot vectors
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
            
        # calculate gradient
        self.dinputs = -y_true / dvalues
        # normalize gradient
        self.dinputs = self.dinputs / samples
        # with a larger number of batches, it's all summed together
        # in the dot product, and some will be given more importance
        # than others when we don't normaize

### Softmax Activation Derivative
We need to find out the impact of one array (the output of the neural network) on another array (the exponentiated, then normalized outputs). We'll calculate the **Jacobian Matrix**

Because the softmax formula is the exponentiated output divided by the sum of all the exponentiated outputs, we use the derivative of the division formula

The math boils down to the partial derivative of the softmax of the j-th output of the i-th batch with respect to the k-th output of the i-th batch is {
    S_i,j * ( 1 - S_i,k)    when j == k
    -S_i,j * S_i,k          when j != k
}

In [36]:
# example sample
softmax_output = [0.7, 0.1, 0.2]

# and shape it as a list of samples
import numpy as np

softmax_output = np.array(softmax_output).reshape(3, 1)
print(softmax_output)

[[0.7]
 [0.1]
 [0.2]]


In [37]:
print(np.eye(softmax_output.shape[0]))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
#print(softmax_output * )